In [1]:
#5 More features split to ensure code can operate

In [2]:
import pandas as pd 
import numpy as np
from joblib import Parallel, delayed
import spacy

nlp=spacy.load('en_core_web_sm')

In [3]:
df= pd.read_parquet(r"C:\Users\brand\UNISA_Honour_Fake_News_Program\Dataset_Merged_BackUp\ForModeling_BagofWords.parquet")
#ForModeling_BagofWords
#ForModeling
df=df.sample(frac=1,random_state=42).head(50000)
df=df.reset_index(drop=True)

df_test=pd.read_parquet(r"C:\Users\brand\UNISA_Honour_Fake_News_Program\Dataset_Merged_BackUp\ForModeling_BagofWords_test.parquet")
df_test=df_test.sample(frac=1,random_state=42)#.head(500)
df_test=df_test.reset_index(drop=True)

In [4]:
last_1000_columns = df.iloc[:, -1000:]
df = df.iloc[:, :-1000]
last_1000_columns.head()

,BOW_abil,BOW_abl,BOW_absolut,BOW_abus,BOW_accept,BOW_access,BOW_accord,BOW_account,BOW_accus,BOW_achiev,...,BOW_worri,BOW_worth,BOW_write,BOW_wrong,BOW_wrote,BOW_ye,BOW_year,BOW_yearold,BOW_york,BOW_young
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,1,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [5]:
last_1000_columns_2= df_test.iloc[:, -1000:]
df_test = df_test.iloc[:, :-1000]
last_1000_columns_2.head()

,BOW_abil,BOW_abl,BOW_absolut,BOW_abus,BOW_accept,BOW_access,BOW_accord,BOW_account,BOW_accus,BOW_achiev,...,BOW_worri,BOW_worth,BOW_write,BOW_wrong,BOW_wrote,BOW_ye,BOW_year,BOW_yearold,BOW_york,BOW_young
0,0,0,0,0,0,0,0,0,1,0,...,0,0,0,1,0,0,0,1,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,1,0
3,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
df.shape

(50000, 13)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 13 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   text                           50000 non-null  object 
 1   label                          50000 non-null  float64
 2   word_count                     50000 non-null  int32  
 3   avg_sentence_length            49998 non-null  float64
 4   text_punctuation_frequency     50000 non-null  int32  
 5   text_capitalization_frequency  50000 non-null  int32  
 6   text_before_character_removal  50000 non-null  int64  
 7   text_after_character_removal   50000 non-null  int64  
 8   text_processed                 50000 non-null  object 
 9   subjectivity_text              50000 non-null  float64
 10  text_sentiment                 50000 non-null  float64
 11  text_reading_ease              50000 non-null  float64
 12  text_lexical_diversity         50000 non-null 

In [8]:
import os

num_cores = os.cpu_count()
print(f"Number of CPU cores: {num_cores}")

Number of CPU cores: 12


In [9]:
from multiprocessing import Pool, cpu_count

from tqdm import tqdm  # Import tqdm for progress tracking


def process_text(text):
    return nlp(text)

def count_entity_tags(doc, entity_label):
    entity_count = sum(1 for ent in doc.ents if ent.label_ == entity_label)
    return entity_count / len(doc) if len(doc) > 0 else 0  # Calculate percentage

def apply_entity_tags(docs, entity_label):
    return [count_entity_tags(doc, entity_label) for doc in docs]

# Assuming df is your DataFrame with 'text_processed' column
texts = df['text_processed'].tolist()

# Process all texts using SpaCy once
processed_docs = []
for text in tqdm(texts, desc="Processing texts", unit="text"):
    processed_docs.append(process_text(text))

# List of SpaCy entity types to calculate percentages for
entity_types = ['ORG', 'NORP', 'GPE', 'PERSON', 'MONEY', 'DATE', 'CARDINAL', 'ORDINAL',
                'LAW', 'PRODUCT', 'EVENT', 'TIME', 'LOC', 'ORG', 'WORK_OF_ART', 'QUANTITY']

# Calculate entity tag percentages for each type and create columns in the DataFrame
for entity_type in tqdm(entity_types, desc="Calculating entities", unit="entity"):
    column_name = f"{entity_type.lower()}_percentage"
    percentages = apply_entity_tags(processed_docs, entity_type)
    df[column_name] = percentages

Calculating entities: 100%|██████████| 16/16 [00:26<00:00,  1.65s/entity]


In [10]:
df.describe()

,label,word_count,avg_sentence_length,text_punctuation_frequency,text_capitalization_frequency,text_before_character_removal,text_after_character_removal,subjectivity_text,text_sentiment,text_reading_ease,...,date_percentage,cardinal_percentage,ordinal_percentage,law_percentage,product_percentage,event_percentage,time_percentage,loc_percentage,work_of_art_percentage,quantity_percentage
count,50000.000000,50000.000000,49998.000000,50000.0,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,...,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000
mean,0.489460,479.009540,164.285894,0.0,4.903020,2796.016920,2731.415820,0.374161,-0.009642,-243.049222,...,0.014627,0.007827,0.002489,0.000111,0.000343,0.000225,0.001183,0.000618,0.000082,0.000091
std,0.499894,578.638519,265.532728,0.0,12.698921,3094.976163,2946.834454,0.134674,0.823429,327.512763,...,0.012829,0.009038,0.005184,0.001465,0.001798,0.001812,0.004089,0.002630,0.000735,0.000765
min,0.000000,0.000000,1.000000,0.0,0.000000,1.000000,1.000000,0.000000,-1.000000,-10710.540000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,183.000000,27.642857,0.0,0.000000,1196.000000,1179.000000,0.302033,-0.924600,-322.350000,...,0.005714,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,344.000000,41.000000,0.0,1.000000,2080.000000,2042.000000,0.377777,0.000000,-167.730000,...,0.012325,0.005952,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,1.000000,581.000000,227.000000,0.0,6.000000,3455.000000,3383.250000,0.449505,0.902200,-71.650000,...,0.020339,0.011338,0.003683,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,25334.000000,10210.000000,0.0,909.000000,142961.000000,111392.000000,1.000000,1.000000,206.840000,...,0.333333,0.250000,0.250000,0.067797,0.166667,0.090909,0.333333,0.136364,0.038462,0.040000


In [11]:
from multiprocessing import Pool, cpu_count

from tqdm import tqdm  # Import tqdm for progress tracking


def process_text(text):
    return nlp(text)

def count_entity_tags(doc, entity_label):
    entity_count = sum(1 for ent in doc.ents if ent.label_ == entity_label)
    return entity_count / len(doc) if len(doc) > 0 else 0  # Calculate percentage

def apply_entity_tags(docs, entity_label):
    return [count_entity_tags(doc, entity_label) for doc in docs]

# Assuming df is your DataFrame with 'text_processed' column
texts = df_test['text_processed'].tolist()

# Process all texts using SpaCy once
processed_docs = []
for text in tqdm(texts, desc="Processing texts", unit="text"):
    processed_docs.append(process_text(text))

# List of SpaCy entity types to calculate percentages for
entity_types = ['ORG', 'NORP', 'GPE', 'PERSON', 'MONEY', 'DATE', 'CARDINAL', 'ORDINAL',
                'LAW', 'PRODUCT', 'EVENT', 'TIME', 'LOC', 'ORG', 'WORK_OF_ART', 'QUANTITY']

# Calculate entity tag percentages for each type and create columns in the DataFrame
for entity_type in tqdm(entity_types, desc="Calculating entities", unit="entity"):
    column_name = f"{entity_type.lower()}_percentage"
    percentages = apply_entity_tags(processed_docs, entity_type)
    df_test[column_name] = percentages

Calculating entities: 100%|██████████| 16/16 [00:06<00:00,  2.34entity/s]


In [12]:
df.shape

(50000, 28)

In [13]:
print(df.describe())

              label    word_count  avg_sentence_length  \
count  50000.000000  50000.000000         49998.000000   
mean       0.489460    479.009540           164.285894   
std        0.499894    578.638519           265.532728   
min        0.000000      0.000000             1.000000   
25%        0.000000    183.000000            27.642857   
50%        0.000000    344.000000            41.000000   
75%        1.000000    581.000000           227.000000   
max        1.000000  25334.000000         10210.000000   

       text_punctuation_frequency  text_capitalization_frequency  \
count                     50000.0                   50000.000000   
mean                          0.0                       4.903020   
std                           0.0                      12.698921   
min                           0.0                       0.000000   
25%                           0.0                       0.000000   
50%                           0.0                       1.000000   
7

In [14]:
df = pd.concat([df, last_1000_columns], axis=1)
df.head()

,text,label,word_count,avg_sentence_length,text_punctuation_frequency,text_capitalization_frequency,text_before_character_removal,text_after_character_removal,text_processed,subjectivity_text,...,BOW_worri,BOW_worth,BOW_write,BOW_wrong,BOW_wrote,BOW_ye,BOW_year,BOW_yearold,BOW_york,BOW_young
0,tactical augmented reality will provide a vide...,0.0,452,21.523810,0,14,2437,2379,tactic augment realiti provid video display so...,0.404365,...,0,0,0,0,0,0,0,0,0,0
1,new york reuters us president donald trump on...,0.0,131,26.200000,0,5,749,735,new york reuter us presid donald trump thursda...,0.727273,...,0,0,0,0,0,0,0,0,1,0
2,stand anthem nt play owner dallas cowboys told...,1.0,168,168.000000,0,0,1124,1124,stand anthem nt play owner dalla cowboy told p...,0.405952,...,0,0,0,0,0,0,0,0,0,0
3,tbilisi reuters kakha kaladze climbed to the ...,0.0,478,23.900000,0,10,2557,2488,tbilisi reuter kakha kaladz climb top world so...,0.448527,...,0,0,0,0,0,0,1,1,0,0
4,washington reuters gary cohn president donald...,0.0,439,29.266667,0,8,2366,2293,washington reuter gari cohn presid donald trum...,0.365613,...,0,0,0,0,0,0,0,0,1,0


In [15]:
df_test = pd.concat([df_test, last_1000_columns_2], axis=1)
df_test.head()

,text,label,word_count,avg_sentence_length,text_punctuation_frequency,text_capitalization_frequency,text_before_character_removal,text_after_character_removal,text_processed,subjectivity_text,...,BOW_worri,BOW_worth,BOW_write,BOW_wrong,BOW_wrote,BOW_ye,BOW_year,BOW_yearold,BOW_york,BOW_young
0,wrong joke macron defend airconditioning gag a...,1,203,203.0,0,0,1494,1494,wrong joke macron defend aircondit gag africa ...,0.274694,...,0,0,0,1,0,0,0,1,0,0
1,trump meet wilbur ross possible choice commerc...,1,75,75.0,0,0,555,555,trump meet wilbur ross possibl choic commerc s...,0.423106,...,0,0,0,0,0,0,0,0,1,0
2,talk trumpism new political journal enters fra...,1,622,622.0,0,0,4433,4433,talk trumpism new polit journal enter fray new...,0.305204,...,0,0,1,0,0,0,1,0,1,0
3,great job wilbur ross destroy proparis accord ...,0,183,183.0,0,0,1296,1296,great job wilbur ross destroy propari accord a...,0.466518,...,0,0,0,0,0,0,1,0,0,0
4,michael moore call day resistance fight trump ...,1,197,197.0,0,0,1398,1398,michael moor call day resist fight trump filmm...,0.401970,...,0,0,0,0,0,0,0,0,0,0


In [16]:


df.to_parquet(r"C:\Users\brand\UNISA_Honour_Fake_News_Program\Dataset_Merged_BackUp\ForModeling_Extra_features.parquet")

df.to_csv(r"C:\Users\brand\UNISA_Honour_Fake_News_Program\Dataset_Visualisation\Visualize_2_More_Features.csv")

df_test.to_parquet(r"C:\Users\brand\UNISA_Honour_Fake_News_Program\Dataset_Merged_BackUp\ForModeling_test_Extra_features.parquet")
